In [3]:
import numpy as np

In [16]:
# задание 1
class Matrix:
    __array_priority__ = 1
    
    def __init__(self, as_nparray: np.ndarray):
        self.shape = as_nparray.shape
        self.as_nparray = as_nparray
        self.__nonzero = np.argwhere(as_nparray != 0)
        
    def __matmul__(self, vector: np.ndarray):
        assert len(vector.shape) == 1, "Vector should be one-dimensional"
        assert self.shape[1] == vector.shape[0],\
            "Vector has a mismatch in its core dimension 0 (size %d is different from %d)" % (vector.shape[0], self.shape[1]) 
        result = np.zeros(self.shape[0], dtype=vector.dtype)
        for i, j in self.__nonzero:
            result[i] += self.as_nparray[i, j] * vector[j]
        return result
    
    def __rmatmul__(self, vector: np.ndarray):
        assert len(vector.shape) == 1, "Vector should be one-dimensional"
        assert self.shape[0] == vector.shape[0],\
            "Vector has a mismatch in its core dimension 0 (size %d is different from %d)" % (vector.shape[0], self.shape[0])
        result = np.zeros(self.shape[1], dtype=vector.dtype)
        for i, j in self.__nonzero:
            result[j] += self.as_nparray[i, j] * vector[i]
        return result

In [17]:
m = np.array([[0, 1, 2], [3, -1, 0], [-2, 0, 4], [3, -7, -8]])
a = np.array([1, 2, 3])
b = np.array([1, 2, 3, 4])
M = Matrix(m)

assert not (m @ a - M @ a).any() 
assert not (b @ m - b @ M).any()

Минимизировать $||Ax - b||$ то же самое, что минимизировать $f(x) = (Ax - b)^T(Ax - b) = xA^TAx-x^TA^Tb-b^tAx+b^Tb = xA^TAx-2b^TAx+b^Tb  $, а нахождение минимума $f$ равносильно решению системы: $A^TAx = A^Tb$

In [29]:
# задание 2
def minimize(A: np.ndarray,
             b: np.ndarray,
             x_0: np.ndarray):
    assert len(A.shape) == 2, "Matrix A should be two-dimensional"
    assert len(b.shape) == 1, "Vector b should be one-dimensional"
    k, n = A.shape
    assert k == b.shape[0],\
        "Vector has a mismatch in its core dimension 0 (size %d is different from %d)" % (b.shape[0], k)
    
    A_T = Matrix(A.T) 
    A = Matrix(A)

    x = x_0
    v = A_T @ (A @ x - b)
    d = v
    v_norm = v @ v
    
    for i in range(n):
        Ad = A_T @ (A @ d)        # O(m)
        alpha = v_norm / (d @ Ad) # O(n)
        x = x - alpha * d         # O(1)
        v = v - alpha * Ad        # O(1)
        v_norm_new = v @ v        # O(n)

        d = v + (v_norm_new / v_norm) * d
        v_norm = v_norm_new
    return x
# полагаем, что n <= m, иначе A^TA вырожденная
# таким образом, итоговая асимптотика O((m + n)n + nn) = O(mn)

In [32]:
A = np.array([[1, 0, 0], [1, 2, 1], [0, 0, 1]])
b = np.array([1, 8, 3])
x_0 = np.zeros(3)
x = minimize(A, b, x_0)
assert np.linalg.norm(x - [1, 2, 3]) < 1e-5